This example shows how to use a subclass of InteractiveViewer. 

It makes precisely the same viewer as the first example, but it's done using a subclass which affords more flexibility (and complexity) than the make_viewer function. 

In [ ]:
try:
    import google.colab
    # We're in Colab
    !pip install git+https://github.com/landoskape/syd.git

except ImportError:
    # Not in Colab
    print("This notebook is designed for Google Colab.")
    print("To install locally, run: pip install syd")

In [2]:
# These are the imports we need for the viewer in this example
import random
import numpy as np
import matplotlib.pyplot as plt
from syd import InteractiveViewer

In [3]:
class WaveViewer(InteractiveViewer):
    def __init__(self, amplitude_limit=1.5):
        # This parameter will affect how the waveforms are colored by the callbacks. 
        self.amplitude_limit = amplitude_limit

        # Add parameters to the viewer in the __init__ method. 
        # The way this works is that you use the add_{parameter_type} function to add a parameter to the viewer. 
        # Each parameter requires a name (e.g. "frequency", "sine_amplitude", etc).
        # Each parameter (except for the button) has a value that you need to set when adding it. 
        # Most parameters have additional required arguments like min_value, max_value, options, etc.
        self.add_float("frequency", value=1.0, min_value=0.1, max_value=5.0)
        self.add_unbounded_float("sine_amplitude", value=1.0)
        self.add_integer("square_amplitude", value=1, min_value=0, max_value=5) # Adding an integer parameter will only allow integers.
        self.add_float("sawtooth_amplitude", value=1.0, min_value=0.1, max_value=2.0)
        self.add_selection("sine_color", value="red", options=["red", "blue", "green", "black"])
        self.add_selection("square_color", value="blue", options=["red", "blue", "green", "black"])
        self.add_selection("sawtooth_color", value="green", options=["red", "blue", "green", "black"])
        self.add_multiple_selection("waveform_type", value=["sine", "square", "sawtooth"], options=["sine", "square", "sawtooth"])
        self.add_boolean("show_legend", value=True)
        self.add_boolean("show_grid", value=True)

        # Let's also make some callbacks so the viewer is a bit more interesting
        # For this first one, if you interact with the show_legend parameter, the update_grid function will be called.
        self.on_change("show_legend", self.update_grid)
        # For this second one, if you interact with the show_grid parameter, the update_legend function will be called.
        self.on_change("show_grid", self.update_legend)

        # We want the color to be updated whenever the amplitude changes of ~any~ of the waveforms, so we can be fancy:
        self.on_change(["sine_amplitude", "square_amplitude", "sawtooth_amplitude"], self.mark_high_amplitude)

        # Add a button for randomizing the colors of the waveforms.
        self.add_button("randomize_colors", label="Randomize Colors", callback=self.randomize_colors)

    # These are callback methods. They'll be called whenever the grid / legend parameters change. 
    def update_grid(self, state):
        showing_details = state["show_legend"]
        self.update_boolean("show_grid", value=showing_details)

    # In this one - we figure out whether the grid is being shown, then update the legend parameter accordingly.
    def update_legend(self, state):
        showing_details = state["show_grid"]
        self.update_boolean("show_legend", value=showing_details)

    def mark_high_amplitude(self, state):
        # Here, we check if the amplitude of each waveform exceeds the limit.
        # If it does, we update the color of that specific waveform to black.
        if state["sine_amplitude"] > self.amplitude_limit:
            self.update_selection("sine_color", value="black")
        if state["square_amplitude"] > self.amplitude_limit:
            self.update_selection("square_color", value="black")
        if state["sawtooth_amplitude"] > self.amplitude_limit:
            self.update_selection("sawtooth_color", value="black")

    def randomize_colors(self, state):
        colors = ["red", "blue", "green"]
        random.shuffle(colors)
        self.update_selection("sine_color", value=colors[0])
        self.update_selection("square_color", value=colors[1])
        self.update_selection("sawtooth_color", value=colors[2])
        

    # Note that when the plot method is defined as a method of the class, 
    # you don't need to call the set_plot() method! 
    def plot(self, state):
        """Plot the waveform based on current parameters."""
        t = np.linspace(0, 2 * np.pi, 1000)

        ymin = float("inf")
        ymax = float("-inf")

        fig, ax = plt.subplots()
        if "sine" in state["waveform_type"]:
            ax.plot(
                t,
                state["sine_amplitude"] * np.sin(state["frequency"] * t),
                color=state["sine_color"],
                label="Sine",
            )
            ymin = min(ymin, -state["sine_amplitude"])
            ymax = max(ymax, state["sine_amplitude"])
        if "square" in state["waveform_type"]:
            ax.plot(
                t,
                state["square_amplitude"] * np.sign(np.sin(state["frequency"] * t)),
                color=state["square_color"],
                label="Square",
            )
            ymin = min(ymin, -state["square_amplitude"])
            ymax = max(ymax, state["square_amplitude"])
        if "sawtooth" in state["waveform_type"]:
            ax.plot(
                t,
                state["sawtooth_amplitude"]
                * (t % (2 * np.pi / state["frequency"]))
                * (state["frequency"] / 2 / np.pi),
                color=state["sawtooth_color"],
                label="Sawtooth",
            )
            ymin = min(ymin, -state["sawtooth_amplitude"])
            ymax = max(ymax, state["sawtooth_amplitude"])

        ax.set_xlabel("Time")
        ax.set_ylabel("Amplitude")
        ax.grid(state["show_grid"])
        ax.set_ylim(ymin * 1.1, ymax * 1.1)
        if state["show_legend"]:
            ax.legend()
        return fig

In [ ]:
# Now we deploy! 
# Since we made a class, we first need to instantiate it. 
# Conveniently, the deploy() method returns the viewer class, so we can do it all at once:
viewer = WaveViewer(amplitude_limit=1.2).deploy(continuous=True)